In [ ]:
%load_ext autoreload
%autoreload 2

import geoviews as gv
import geoviews.feature as gf
import xarray as xr
from cartopy import crs as ccrs

import pygplates
from gprm.utils.create_gpml import gpml2gdf
from gprm.datasets import Reconstructions

gv.extension('bokeh', 'matplotlib')



In [ ]:
M2021 = Reconstructions.fetch_Merdith2021()


In [ ]:
Continents = M2021.polygon_snapshot('continents', 750)


In [ ]:
def wrap_features(features, central_longitude = 0., tesselation_degrees = 1.):

    date_line_wrapper = pygplates.DateLineWrapper(central_meridian=central_longitude)

    wrapped_features = []
    for polygon in features:
        geometry = polygon.get_reconstructed_geometry()
        #for geometry in r.get_geometries():
        split_polygons = date_line_wrapper.wrap(geometry, tesselation_degrees)
        for split_polygon in split_polygons:
            #print(split_polygon)
            f = pygplates.Feature()
            f.set_geometry(pygplates.PolygonOnSphere(
                (wrapped_point.get_latitude(), wrapped_point.get_longitude()) for wrapped_point in split_polygon.get_exterior_points())
                          )
            f.set_name(polygon.get_feature().get_name())
            f.set_reconstruction_plate_id(polygon.get_feature().get_reconstruction_plate_id())
            f.set_valid_time(polygon.get_feature().get_valid_time()[0], polygon.get_feature().get_valid_time()[1])
            wrapped_features.append(f)

    return gpml2gdf(wrapped_features)


In [ ]:
dim = 300
gv.Polygons(wrap_features(Continents.reconstructed_polygons), 
            vdims=['NAME', 'PLATEID1', 'FROMAGE']).opts(
    tools=['hover'], height=dim, width=int(dim*1.9), projection=ccrs.Robinson(central_longitude=central_longitude), color_index='FROMAGE'
)